In [1]:
# confident edges: edges that iterative approach predicted early (in other words, "confidently")
# we compare our method and LR method on the accuracy of those edges


import _pickle as pkl
import networkx as nx
import numpy as np
import random

from tqdm import tqdm
from snpp.cores.lowrank import alq_spark, predict_signs
from snpp.utils.matrix import split_train_test, load_sparse_csr
from snpp.utils.signed_graph import g2m
from snpp.utils.data import load_train_test_graphs
from snpp.utils.edge_filter import filter_by_min_triangle_count

from snpp.utils.spark import sc

dataset = 'slashdot'
lambda_ = 0.2
k = 40
max_iter = 100
random_seed = 123456
min_tri_count = 20

recache_input = False

random.seed(random_seed)
np.random.seed(random_seed)

In [3]:
train_g, test_g = load_train_test_graphs(dataset, recache_input)


loading train and test graphs...


In [4]:
################
## DEBUG
#################
print(train_g.number_of_nodes())
print(train_g.number_of_edges())
print(test_g.number_of_edges())
assert train_g.number_of_nodes() == test_g.number_of_nodes()

77357
464917
51658


In [5]:
train_g_ud = train_g.to_undirected()

train_m = g2m(train_g)
train_m_ud = g2m(train_g_ud)
print(train_m_ud.shape)
print(train_m_ud.nnz)

100%|██████████| 77357/77357 [00:01<00:00, 55293.44it/s]


(77357, 77357)
929435


In [16]:
confident_edges = set(filter_by_min_triangle_count(train_g_ud, test_g.edges(), min_tri_count))
print(len(confident_edges))

2645


In [17]:
X, Y = alq_spark(train_m, k=k, sc=sc,
                 lambda_=lambda_, iterations=max_iter,
                 seed=random_seed)

In [24]:
truth = set((i, j, test_g[i][j]['sign']) for i, j in confident_edges)
preds = predict_signs(X, Y, confident_edges, sc)
assert len(truth) == len(preds)
assert set((i, j) for i, j, _ in preds) == set((i ,j) for i, j, _ in truth)

print('=> accuracy on {} edges {} (lowrank)'.format(len(preds), len(truth.intersection(preds)) / len(truth)))

=> accuracy on 2645 edges 0.8480151228733459 (lowrank)


In [23]:
## Iterative approach
from snpp.cores.joint_part_pred import iterative_approach
from snpp.cores.max_balance import faster_greedy
from snpp.cores.lowrank import partition_graph
from snpp.cores.budget_allocation import constant_budget
from snpp.cores.triangle import build_edge2edges

part, iter_preds, status = iterative_approach(
    train_g_ud,
    T=confident_edges,
    k=k,
    graph_partition_f=partition_graph,
    graph_partition_kwargs=dict(sc=sc,
                                lambda_=lambda_,
                                iterations=max_iter,
                                seed=random_seed),
    budget_allocation_f=constant_budget,
    budget_allocation_kwargs=dict(const=200),
    solve_maxbalance_f=faster_greedy,
    solve_maxbalance_kwargs={'edge2edges': build_edge2edges(train_g_ud.copy(),
                                                            confident_edges)},
    truth=set([(i, j, test_g[i][j]['sign'])
               for i, j in confident_edges]),
    perform_last_partition=False)


  6%|▌         | 147/2645 [00:00<00:01, 1469.33it/s]

build edge2edges


100%|██████████| 2645/2645 [00:01<00:00, 1839.39it/s]


iteration=1, #remaining targets=2645
graph partitioning...
to_scipy_sparse_matrix


100%|██████████| 77357/77357 [00:01<00:00, 48933.31it/s]


...

solving max_balance
building triangle_count_by_edge
edge2edges (size 2534) is given
assigning 1 to (2775, 1808) produces 15 more balanced triangles (3, 18): (('s+1', 18), ('s-1', 3))
assigning 1 to (1808, 12385) produces 15 more balanced triangles (2, 17): (('s+1', 17), ('s-1', 2))
assigning 1 to (1808, 2775) produces 15 more balanced triangles (3, 18): (('s+1', 18), ('s-1', 3))
assigning 1 to (1200, 3143) produces 15 more balanced triangles (2, 17): (('s+1', 17), ('s-1', 2))
assigning -1 to (12320, 523) produces 14 more balanced triangles (17, 3): (('s-1', 13), ('w-1', 4), ('s+1', 3))
assigning -1 to (2152, 1735) produces 13 more balanced triangles (25, 12): (('s-1', 24), ('s+1', 12), ('w-1', 1))
assigning -1 to (2152, 632) produces 13 more balanced triangles (16, 3): (('s-1', 16), ('s+1', 3))
assigning -1 to (5994, 4612) produces 13 more balanced triangles (16, 3): (('s-1', 15), ('s+1', 3), ('w-1', 1))
assigning -1 to (2152, 8390) produces 13 more balanced triangles (18, 5): (('w-1',

100%|██████████| 77357/77357 [00:01<00:00, 71809.40it/s]


ALS...
predict labels (SVD + Kmeans)...
eigen values: [ 190.62587012   90.8189479    70.23162142   47.85687933   35.12667946
   32.65797912   28.23341359   26.69860891   26.16669271   25.72444006
   25.19341162   24.60485371   24.06378406   22.91160148   22.10615927
   21.0816226    20.65910164   20.20905439   19.3345529    19.19318564
   18.7026865    17.89293773   17.33833806   16.64951322   16.15714751
   14.92807364   14.37732234   13.96198055   13.65580721   13.06731072
   12.10263204   11.79630655   11.06519575   10.6029487    10.35380168
   10.05402302    9.42594815    9.14825099    8.81439394    8.45954117]


/home/cloud-user/code/snpp/venv/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/cloud-user/code/snpp/venv/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/cloud-user/code/snpp/venv/lib/python3.5/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T

solving max_balance
building triangle_count_by_edge
edge2edges (size 2620) is given
assigning -1 to (2099, 1325) produces 7 more balanced triangles (14, 7): (('s-1', 10), ('s+1', 7), ('w-1', 4))
assigning -1 to (4072, 523) produces 6 more balanced triangles (13, 7): (('s-1', 8), ('s+1', 7), ('w-1', 5))
assigning 1 to (1287, 3418) produces 5 more balanced triangles (6, 11): (('s+1', 11), ('s-1', 6))
assigning 1 to (3393, 3335) produces 4 more balanced triangles (16, 20): (('s+1', 20), ('w-1', 10), ('s-1', 6))
assigning 1 to (1092, 901) produces 3 more balanced triangles (9, 12): (('s+1', 12), ('s-1', 9))
assigning -1 to (5135, 170) produces 3 more balanced triangles (10, 7): (('s+1', 7), ('w-1', 5), ('s-1', 5))
assigning 1 to (669, 78) produces 3 more balanced triangles (13, 16): (('s+1', 16), ('s-1', 9), ('w-1', 4))
assigning 1 to (1287, 7497) produces 2 more balanced triangles (10, 12): (('s+1', 12), ('s-1', 10))
assigning -1 to (3096, 3173) produces 2 more balanced triangles (13, 11)